# Test Best Models

Test notebook to generate the statistics of the different models found with AMS. First load the required libraries

In [1]:
import datetime
import logging
import sys
import numpy as np

from sklearn.preprocessing import StandardScaler, MinMaxScaler

from keras.callbacks import LearningRateScheduler
from keras.optimizers import Adam
import keras.backend as K
from keras.models import model_from_json

from sklearn.model_selection import KFold
from scipy import stats

#sys.path.append('/Users/davidlaredorazo/Documents/University_of_California/Research/Projects')
sys.path.append('/media/controlslab/DATA/Projects')

import ann_framework.aux_functions as aux_functions

import automatic_model_selection
from automatic_model_selection import Configuration
from ann_encoding_rules import Layers
import fetch_to_keras
#from CMAPSAuxFunctions import TrainValTensorBoard

#Tunable model
from ann_framework.tunable_model.tunable_model import SequenceTunableModelRegression, SequenceTunableModelClassification

#Data handlers
from ann_framework.data_handlers.data_handler_CMAPSS import CMAPSSDataHandler
from ann_framework.data_handlers.data_handler_MNIST import MNISTDataHandler
from ann_framework.data_handlers.data_handler_CIFAR10 import CIFAR10DataHandler

Using TensorFlow backend.


### Given a model, get the compiled model

In [2]:
def get_compiled_model(model, problem_type, optimizer_params=[]):
    """Obtain a keras compiled model"""
    
    #Shared parameters for the models
    optimizer = Adam(lr=0.001, beta_1=0.5)
    
    if problem_type == 1:
        lossFunction = "mean_squared_error"
        metrics = ["mse"]
    elif problem_type == 2:
        lossFunction = "categorical_crossentropy"
        metrics = ["accuracy"]
    else:
        print("Problem type not defined")
        model = None
        return
    
    #Create and compile the models
    model.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)
    
    return model


def create_tunable_model(model_genotype, problem_type, input_shape, data_handler, model_number):
    
    K.clear_session()
    
    model = fetch_to_keras.decode_genotype(model_genotype, problem_type, input_shape, 1)
    
    model = get_compiled_model(model, problem_type, optimizer_params=[])
    
    if problem_type == 1:
        tModel = SequenceTunableModelRegression('ModelReg_SN_'+str(model_number), model, lib_type='keras', data_handler=data_handler)
    else:
        tModel = SequenceTunableModelClassification('ModelClass_SN_'+str(model_number), model, lib_type='keras', data_handler=data_handler)
        
    return tModel

### Load cmaps data handler

In [3]:
def cmaps_dhandler(data_scaler=None):

    #Selected as per CNN paper
    features = ['T2', 'T24', 'T30', 'T50', 'P2', 'P15', 'P30', 'Nf', 'Nc', 'epr', 'Ps30', 'phi', 'NRf', 'NRc', 'BPR', 
    'farB', 'htBleed', 'Nf_dmd', 'PCNfR_dmd', 'W31', 'W32']
    selected_indices = np.array([2, 3, 4, 7, 8, 9, 11, 12, 13, 14, 15, 17, 20, 21])
    selected_features = list(features[i] for i in selected_indices-1)
    data_folder = '../CMAPSSData'

    window_size = 24
    window_stride = 1
    max_rul = 129

    dHandler_cmaps = CMAPSSDataHandler(data_folder, 1, selected_features,
                                       max_rul, window_size, window_stride, data_scaler=data_scaler)

    input_shape = (len(selected_features)*window_size, )

    return dHandler_cmaps

### Load models and evaluate model

In [4]:
def load_model(model_file, weights_file="", problem_type=1):
    
    p_type = ""
    
    # load json and create model
    json_file = open(model_file, 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    print("Loaded model from disk") 
        
    #Load weights if specified
    if weights_file != "":
        # load weights into new model
        loaded_model.load_weights(weights_file)
        print("Loaded weights from disk") 
    else:
        print("Model needs training")
        
    optimizer = Adam(lr=0.001, beta_1=0.5)
    
    if problem_type == 1:
        p_type = "regression"
        lossFunction = "mean_squared_error"
        metrics = ["mse"]
    elif problem_type == 2:
        p_type = "classification"
        lossFunction = "categorical_crossentropy"
        metrics = ["accuracy"]
    else:
        print("Problem type not defined")
        model = None
        return
    
    #Create and compile the models
    loaded_model.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)
    print("Created model for " + p_type + " with loss function " + lossFunction)

    return loaded_model
        
        

### Load each of the models and test them

In [5]:
best_model_folder = 'best_models'
"""
test_sets = {'cifar10':(CIFAR10DataHandler, None, 2), 
             'cmapss':(cmaps_dhandler, MinMaxScaler(feature_range=(-1, 1)), 1), 
             'mnist':((MNISTDataHandler), None, 2)}
"""

test_sets = {'cmapss':(cmaps_dhandler, MinMaxScaler(feature_range=(-1, 1)), 1)}
#alpha_folders = ['alpha0.6', 'alpha0.8', 'alpha1']
alpha_folders = ['alpha0.6']

In [6]:
results = {}

k = 10  #For 10-fold cross validation

model_file_name = 'smaller_bestModel_global.json'
weights_file_name = 'smaller_bestModel_global.h5'

for dataset in test_sets.keys():
    for alphaValue in alpha_folders:
        
        results_key = dataset + "_" +  alphaValue
        
        evaluations_cv = np.zeros(k)
        evaluations_test = np.zeros(k)
        
        model_location = best_model_folder + '/' + dataset + '/' + alphaValue + '/' + model_file_name
        
        if weights_file_name != "":
            weights_location = best_model_folder + '/' + dataset + '/' + alphaValue + '/' + weights_file_name
        
        dhandler, data_scaler, problem_type = test_sets[dataset]

        #model = load_model(model_location, weights_location, problem_type)   
        
        data_handler = dhandler(data_scaler=data_scaler)
        data_handler.load_data(verbose = 1, )
        data_handler.print_data()
        
        folds = list(KFold(n_splits=k, shuffle=True).split(data_handler.X_train))
        
        print('Validation on model:' + model_location)
        
        for j, (train_idx, val_idx) in enumerate(folds):

            print('\nExperiment on Fold ', j)
            
            K.clear_session()  #Clear the previous tensorflow graph 

            X_train_cv = data_handler.X_train[train_idx]
            y_train_cv = data_handler.y_train[train_idx]
            X_valid_cv = data_handler.X_train[val_idx]
            y_valid_cv = data_handler.y_train[val_idx]

            model = load_model(model_location, "", problem_type)
            model.summary()

            model.fit(X_train_cv, y_train_cv, batch_size=512, epochs=100, verbose=1)

            evaluation_cv = model.evaluate(X_valid_cv, y_valid_cv)
            evaluation_test = model.evaluate(data_handler.X_test, data_handler.y_test)

            evaluations_cv[j] = evaluation_cv[1]
            evaluations_test[j] = evaluation_test[1]
            
        results[results_key] = (evaluations_cv, evaluations_test)
        

Loading data for dataset 1 with window_size of 24, stride of 1 and maxRUL of 129. Cros-Validation ratio 0
Loading data from file and computing dataframes


/home/controlslab/anaconda3/envs/tf_gpu/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


Printing shapes

Training data (X, y)
(18331, 336)
(18331, 1)
Testing data (X, y)
(100, 336)
(100, 1)
Printing first 5 elements

Training data (X, y)
[[-0.63253012 -0.18639634 -0.38048616 ... -0.33333333  0.33333333
   0.31289699]
 [-0.43373494 -0.09396119 -0.29473329 ... -0.16666667  0.25581395
   0.47638774]
 [-0.31325301 -0.26095487 -0.25894666 ...  0.          0.11627907
   0.43800055]
 [-0.31325301 -0.48768258 -0.33760972 ... -0.16666667  0.31782946
   0.52720243]
 [-0.30120482 -0.48506649 -0.19074949 ... -0.66666667  0.34883721
   0.07677437]]
[[129.]
 [129.]
 [129.]
 [129.]
 [129.]]
Testing data (X, y)
[[-0.19879518 -0.5705254  -0.37069548 ... -0.16666667  0.03875969
   0.27312897]
 [-0.21686747 -0.30237628 -0.12086428 ... -0.5         0.03875969
   0.01518917]
 [-0.04216867 -0.1942446  -0.0209318  ...  0.16666667  0.2248062
   0.04888152]
 [-0.22891566 -0.01722259 -0.13673194 ...  0.16666667 -0.31782946
   0.004971  ]
 [-0.11445783 -0.11968607  0.03511141 ...  0.         -0.054

16497/16497 [==============================] - 0s 3us/step - loss: 272.0772 - mean_squared_error: 272.0772
Epoch 53/100
16497/16497 [==============================] - 0s 3us/step - loss: 271.6109 - mean_squared_error: 271.6109
Epoch 54/100
16497/16497 [==============================] - 0s 3us/step - loss: 268.9988 - mean_squared_error: 268.9988
Epoch 55/100
16497/16497 [==============================] - 0s 3us/step - loss: 271.1783 - mean_squared_error: 271.1783
Epoch 56/100
16497/16497 [==============================] - 0s 3us/step - loss: 270.4957 - mean_squared_error: 270.4957
Epoch 57/100
16497/16497 [==============================] - 0s 3us/step - loss: 267.7562 - mean_squared_error: 267.7562
Epoch 58/100
16497/16497 [==============================] - 0s 3us/step - loss: 269.2036 - mean_squared_error: 269.2036
Epoch 59/100
16497/16497 [==============================] - 0s 3us/step - loss: 266.3713 - mean_squared_error: 266.3713
Epoch 60/100
16497/16497 [===========================

16498/16498 [==============================] - 0s 3us/step - loss: 357.3559 - mean_squared_error: 357.3559
Epoch 13/100
16498/16498 [==============================] - 0s 3us/step - loss: 346.9425 - mean_squared_error: 346.9425
Epoch 14/100
16498/16498 [==============================] - 0s 3us/step - loss: 341.0681 - mean_squared_error: 341.0681
Epoch 15/100
16498/16498 [==============================] - 0s 3us/step - loss: 331.6797 - mean_squared_error: 331.6797
Epoch 16/100
16498/16498 [==============================] - 0s 3us/step - loss: 330.7598 - mean_squared_error: 330.7598
Epoch 17/100
16498/16498 [==============================] - 0s 3us/step - loss: 326.3490 - mean_squared_error: 326.3490
Epoch 18/100
16498/16498 [==============================] - 0s 3us/step - loss: 319.6550 - mean_squared_error: 319.6550
Epoch 19/100
16498/16498 [==============================] - 0s 3us/step - loss: 318.0134 - mean_squared_error: 318.0134
Epoch 20/100
16498/16498 [===========================

...

16498/16498 [==============================] - 0s 3us/step - loss: 271.0398 - mean_squared_error: 271.0398
Epoch 46/100
16498/16498 [==============================] - 0s 3us/step - loss: 268.3354 - mean_squared_error: 268.3354
Epoch 47/100
16498/16498 [==============================] - 0s 3us/step - loss: 267.7060 - mean_squared_error: 267.7060
Epoch 48/100
16498/16498 [==============================] - 0s 3us/step - loss: 265.9534 - mean_squared_error: 265.9534
Epoch 49/100
16498/16498 [==============================] - 0s 3us/step - loss: 268.4971 - mean_squared_error: 268.4971
Epoch 50/100
16498/16498 [==============================] - 0s 3us/step - loss: 264.5813 - mean_squared_error: 264.5813
Epoch 51/100
16498/16498 [==============================] - 0s 3us/step - loss: 266.2072 - mean_squared_error: 266.2072
Epoch 52/100
16498/16498 [==============================] - 0s 3us/step - loss: 264.1098 - mean_squared_error: 264.1098
Epoch 53/100
16498/16498 [===========================

16498/16498 [==============================] - 0s 3us/step - loss: 521.4786 - mean_squared_error: 521.4786
Epoch 6/100
16498/16498 [==============================] - 0s 3us/step - loss: 486.3728 - mean_squared_error: 486.3728
Epoch 7/100
16498/16498 [==============================] - 0s 3us/step - loss: 457.3591 - mean_squared_error: 457.3591
Epoch 8/100
16498/16498 [==============================] - 0s 3us/step - loss: 430.3209 - mean_squared_error: 430.3209
Epoch 9/100
16498/16498 [==============================] - 0s 3us/step - loss: 410.3468 - mean_squared_error: 410.3468
Epoch 10/100
16498/16498 [==============================] - 0s 3us/step - loss: 391.2173 - mean_squared_error: 391.2173
Epoch 11/100
16498/16498 [==============================] - 0s 3us/step - loss: 376.0158 - mean_squared_error: 376.0158
Epoch 12/100
16498/16498 [==============================] - 0s 3us/step - loss: 360.5974 - mean_squared_error: 360.5974
Epoch 13/100
16498/16498 [==============================]

16498/16498 [==============================] - 0s 3us/step - loss: 255.4983 - mean_squared_error: 255.4983
Epoch 74/100
16498/16498 [==============================] - 0s 3us/step - loss: 254.9084 - mean_squared_error: 254.9084
Epoch 75/100
16498/16498 [==============================] - 0s 3us/step - loss: 255.2372 - mean_squared_error: 255.2372
Epoch 76/100
16498/16498 [==============================] - 0s 3us/step - loss: 253.7994 - mean_squared_error: 253.7994
Epoch 77/100
16498/16498 [==============================] - 0s 3us/step - loss: 253.2001 - mean_squared_error: 253.2001
Epoch 78/100
16498/16498 [==============================] - 0s 3us/step - loss: 252.2451 - mean_squared_error: 252.2451
Epoch 79/100
16498/16498 [==============================] - 0s 3us/step - loss: 253.0426 - mean_squared_error: 253.0426
Epoch 80/100
16498/16498 [==============================] - 0s 3us/step - loss: 252.0117 - mean_squared_error: 252.0117
Epoch 81/100
16498/16498 [===========================

16498/16498 [==============================] - 0s 3us/step - loss: 293.5899 - mean_squared_error: 293.5899
Epoch 34/100
16498/16498 [==============================] - 0s 3us/step - loss: 289.6793 - mean_squared_error: 289.6793
Epoch 35/100
16498/16498 [==============================] - 0s 3us/step - loss: 292.7963 - mean_squared_error: 292.7963
Epoch 36/100
16498/16498 [==============================] - 0s 3us/step - loss: 290.0775 - mean_squared_error: 290.0775
Epoch 37/100
16498/16498 [==============================] - 0s 3us/step - loss: 288.1872 - mean_squared_error: 288.1872
Epoch 38/100
16498/16498 [==============================] - 0s 3us/step - loss: 288.8531 - mean_squared_error: 288.8531
Epoch 39/100
16498/16498 [==============================] - 0s 3us/step - loss: 286.4853 - mean_squared_error: 286.4853
Epoch 40/100
16498/16498 [==============================] - 0s 3us/step - loss: 286.3503 - mean_squared_error: 286.3503
Epoch 41/100
16498/16498 [===========================

100/100 [==============================] - 0s 20us/step


### Print the stats

In [7]:
for key in results.keys():
    
    print("\nStats for: ")
    print(key)
    evaluations_cv, evaluations_test = results[key]
    
    print("\nCrossVal stats: ")
    print(evaluations_cv)
    print(stats.describe(evaluations_cv))
    
    print("\nTest stats: ")
    print(stats.describe(evaluations_test))


Stats for: 
cmapss_alpha0.6

CrossVal stats: 
[233.77221355 259.64944601 242.45230557 228.72101412 259.46522168
 234.58927604 224.83541721 236.86602743 233.40204024 218.30087841]
DescribeResult(nobs=10, minmax=(218.30087840927797, 259.6494460118903), mean=237.20538402789435, variance=182.48429868361785, skewness=0.6227328361273433, kurtosis=-0.5652334255902991)

Test stats: 
DescribeResult(nobs=10, minmax=(220.61275451660157, 246.84741088867187), mean=233.08709432983397, variance=64.79279867555337, skewness=0.12078297786194936, kurtosis=-0.8721861692821378)
